# NeuralQuant

##### Desenvolvido por <b>Felipe Santana Viana</b>

##### Importando bibliotecas

In [1]:
from requests import get
from zipfile import ZipFile
from io import BytesIO
from numpy import array, reshape
from pandas import read_csv, concat, DataFrame
from pandas_datareader import DataReader
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from scipy.stats import pearsonr
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from csv import DictWriter
from statsmodels.api import OLS, add_constant
import matplotlib.pyplot as plt

##### Função para escrever arquivos CSV (útil em manutenção do histórico em caso de erros)

In [3]:
def write_csv(file_path, fieldnames):
    '''Função para escrever arquivos CSV (útil em manutenção do histórico em caso de erros)'''
    header_added = False
    fieldnames_list = []
    for i in fieldnames.keys():
        fieldnames_list.append(i)
    with open(file_path, 'a', newline='') as csvfile:
        writer = DictWriter(csvfile, fieldnames=fieldnames_list)
        def writerow_func():
            return writer.writerow(fieldnames)
        try:
            df = read_csv(file_path)
            if list(fieldnames.keys())[0] in df.columns:
                writerow_func()
            else:
                writer.writeheader()
                writerow_func()
        except:
            writer.writeheader()
            writerow_func()
    return True

##### Importando a carteira do Ibovespa de set. a dez. de 2022

In [2]:
ibov_comp = read_csv('resources/IBOVQuad_9-2022.csv')

##### Adicionando ".SA" no ticker para consultar no Yahoo Finance

In [3]:
tickers = []
for t in list(ibov_comp['codigo']):
    tickers.append(t+'.SA')

##### Consultando a quantidade de registros para treino de cada ação e salvando no arquivo "outputs/tickers.csv"

In [5]:
for t in tickers:
    try:
        inicio = datetime(2010,1,1)
        final = datetime(2020,1,1)
        data = DataReader(t, 'yahoo', inicio, final)['Adj Close']
        qtd_registros_treino = len(data.dropna())
        
        write_csv('outputs/tickers.csv', {'ticker': t, 'qtd_registros_treino': qtd_registros_treino, 'fl_success': 1})
        print(t + ': Sucesso')
    except:
        write_csv('outputs/tickers.csv', {'ticker': t, 'qtd_registros_treino': None, 'fl_success': 0})
        print(t + ': Erro')

ABEV3.SA: Sucesso
ALPA4.SA: Sucesso
AMER3.SA: Sucesso
ARZZ3.SA: Sucesso
ASAI3.SA: Erro
AZUL4.SA: Sucesso
B3SA3.SA: Sucesso
BBAS3.SA: Sucesso
BBDC3.SA: Sucesso
BBDC4.SA: Sucesso
BBSE3.SA: Sucesso
BEEF3.SA: Sucesso
BPAC11.SA: Sucesso
BPAN4.SA: Sucesso
BRAP4.SA: Sucesso
BRFS3.SA: Sucesso
BRKM5.SA: Sucesso
BRML3.SA: Sucesso
CASH3.SA: Erro
CCRO3.SA: Sucesso
CIEL3.SA: Sucesso
CMIG4.SA: Sucesso
CMIN3.SA: Erro
COGN3.SA: Sucesso
CPFE3.SA: Sucesso
CPLE6.SA: Sucesso
CRFB3.SA: Sucesso
CSAN3.SA: Sucesso
CSNA3.SA: Sucesso
CVCB3.SA: Sucesso
CYRE3.SA: Sucesso
DXCO3.SA: Sucesso
ECOR3.SA: Sucesso
EGIE3.SA: Sucesso
ELET3.SA: Sucesso
ELET6.SA: Sucesso
EMBR3.SA: Sucesso
ENBR3.SA: Sucesso
ENEV3.SA: Sucesso
ENGI11.SA: Sucesso
EQTL3.SA: Sucesso
EZTC3.SA: Sucesso
FLRY3.SA: Sucesso
GGBR4.SA: Sucesso
GOAU4.SA: Sucesso
GOLL4.SA: Sucesso
HAPV3.SA: Sucesso
HYPE3.SA: Sucesso
IGTI11.SA: Erro
IRBR3.SA: Sucesso
ITSA4.SA: Sucesso
ITUB4.SA: Sucesso
JBSS3.SA: Sucesso
KLBN11.SA: Sucesso
LREN3.SA: Sucesso
LWSA3.SA: Erro
MGL

##### Lendo o arquivo "outputs/tickers.csv" salvo na etapa anterior, filtrando aqueles com sucesso, tirando resultados duplicados e transfornando em lista

In [2]:
tickers_sucesso_pre = read_csv('outputs/tickers.csv')
tickers_sucesso = tickers_sucesso_pre[tickers_sucesso_pre['fl_success'] == 1]
tickers_sucesso_unico = list(set(tickers_sucesso['ticker']))

##### Treinamento das Redes Neurais

###### Obs: parte de inteligência do código omitida. Entre em contato com felipesantana76@gmail.com para mais detalhes.

In [4]:
for t in sorted(tickers_sucesso_unico):
    try:
        inicio = datetime(2010,1,1)
        final = datetime(2020,1,1)

        dados = DataReader(t, 'yahoo', inicio, final)

        normalizando = MinMaxScaler(feature_range=(0,1))

        
        ###############################################################################################################
        ###############################################################################################################
        # Parte de inteligência do código omitida. Entre em contato com felipesantana76@gmail.com para mais detalhes. #
        ###############################################################################################################
        ###############################################################################################################
        

        # Salvando gráficos
        plt.plot(precos_reais, color ='red', label = 'Preço de Fechamento Ajustado Real (R$)')
        plt.plot(previsao_precos, color="black", label = 'Preço de Fechamento Ajustado Previsto (R$)')
        plt.title(t)
        plt.xlabel('Tempo (dias de pregão)')
        plt.ylabel('Preço de Fechamento Ajustado (R$)')
        plt.legend()
        plt.savefig('images/test_graphs/'+t+'.png')
        plt.close()
        
        y_true = precos_reais
        y_pred = previsao_precos
        
        df_true = DataFrame(y_true)
        df_pred = DataFrame(y_pred)
        
        df_join = concat([df_true, df_pred], axis=1)
        df_join.columns = ['y_true', 'y_pred']
        df_join.to_csv('outputs/timeseries/'+t+'.csv', index=False)
        
        model_sm = OLS(y_pred, add_constant(y_true)).fit()
        
        correl = float(pearsonr(y_true, y_pred)[0])
        MSE = mean_squared_error(y_true, y_pred)
        RMSE = sqrt(mean_squared_error(y_true, y_pred))
        MAE = mean_absolute_error(y_true, y_pred)
        MAPE = mean_absolute_percentage_error(y_true, y_pred)
        R2 = model_sm.rsquared
        P_VALOR = model_sm.pvalues[0]
        
        if max(y_true) > max(y_pred):
            max_ = max(y_true)
        else:
            max_ = max(y_pred)

        if min(y_true) < min(y_pred):
            min_ = min(y_true)
        else:
            min_ = min(y_pred)
        
        plt.figure(figsize=(5,5))
        plt.xlim(min_, max_)
        plt.ylim(min_, max_)
        plt.plot([min_, max_], [min_, max_], color='black')
        plt.title(t)
        plt.grid()
        plt.xlabel('Preço de Fechamento Ajustado Real (R$)')
        plt.ylabel('Preço de Fechamento Ajustado Previsto (R$)')
        plt.scatter(y_true, y_pred, color = 'grey', s=3)
        plt.savefig('images/scatter/'+t+'.png')
        plt.close()
        
        plt.rc('figure', figsize=(12, 7))
        plt.text(0.01, 0.05, str(model_sm.summary()), {'fontsize': 10}, fontproperties = 'monospace')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig('outputs/statsmodel_results/'+t+'.png')
        plt.close()
        
        write_csv('outputs/metrics.csv', {'ticker': t, 'CORRELACAO_PEARSON':correl, 'MSE':MSE, 'RMSE':RMSE, 'MAE':MAE, 'MAPE':MAPE, 'R2_SK':R2_SK, 'R2_SM':R2_SM, 'P_VALOR':P_VALOR, 'fl_success': 1})
        print(t + ': Sucesso')
    except:
        write_csv('outputs/metrics.csv', {'ticker': t, 'CORRELACAO_PEARSON':None, 'MSE':None, 'RMSE':None, 'MAE':None, 'MAPE':None, 'R2_SK':None, 'R2_SM':None, 'P_VALOR':None, 'fl_success': 0})
        print(t + ': Erro')


Epoch 1/25
76/76 [==============================] - 20s 122ms/step - loss: 0.0227
Epoch 2/25
76/76 [==============================] - 9s 118ms/step - loss: 0.0041
Epoch 3/25
76/76 [==============================] - 10s 131ms/step - loss: 0.0036
Epoch 4/25
76/76 [==============================] - 9s 121ms/step - loss: 0.0033
Epoch 5/25
76/76 [==============================] - 9s 119ms/step - loss: 0.0031
Epoch 6/25
76/76 [==============================] - 10s 130ms/step - loss: 0.0033
Epoch 7/25
76/76 [==============================] - 9s 124ms/step - loss: 0.0030
Epoch 8/25
76/76 [==============================] - 10s 127ms/step - loss: 0.0029
Epoch 9/25
76/76 [==============================] - 10s 126ms/step - loss: 0.0026
Epoch 10/25
76/76 [==============================] - 10s 128ms/step - loss: 0.0024
Epoch 11/25
76/76 [==============================] - 12s 152ms/step - loss: 0.0032
Epoch 12/25
76/76 [==============================] - 9s 124ms/step - loss: 0.0029
Epoch 13/25
76/76 